# 13_딥러닝(1)
## - 딥러닝으로 XOR 문제 해결 -

XOR 문제 - 딥러닝 아키텍처

- => 딥러닝에서는, 1개 이상의 은닉 층(hidden layer)을 만들 수 있고, 각 은닉 층(hidden layer)에 존재하는 노드(node) 개수 또한 임의의 개수를 만들 수 있음. 그러나 은닉 층과 노드 수가 많아지면 학습 속도가 느려지므로 적절한 개수의 은닉 층과 노드 수를 고려하여 구현하는 것이 필요함.

LogicGate class - 딥러닝 버전

- external function  
  
- def sigmoid(x): # 0 또는 1을 출력하기 위한 sigmoid 함수  

- def numerical_derivative(f, x): # 수치미분함수  

- LogicGate class

- class LogicGate:
    def __init__(self, gate_name, xdata, tdata) # 입력/정답 데이터/가중치/바이어스 초기화  
    def feed_forward(self) # feed forward 이용하여 손실함수 값 계산  
    def error_val(self) # 손실함수 값 계산(외부 출력을 위해 사용됨)  
    def train(self) # 수치미분을 이용하여 손실함수 최소값 찾는 method  
    def predict(self, xdata) # 미래 값 예측 method  

- usage
- xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) # 입력 데이터 생성  
  tdata = np.array([0, 1, 1, 0]) # 정답 데이터 생성 (XOR 예시)  
  
  AND_obj = LogicGate("XOR_GATE", xdata, tdata) # LogicGate 객체생성
  AND_obj.train() # 손실함수 최소값 갖도록 학습
  
  AND_obj.predict(...) # 임의 데이터에 대한 결과 예측

구현코드 - 딥러닝버전

In [9]:
# [1] external function (sigmoid, numerical_derivative)
import numpy as np

# sigmoid 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x)
        
        x[idx] = tmp_val - delta_x
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2 * delta_x)
        
        x[idx] = tmp_val
        it.iternext()
        
    return grad

In [14]:
# [2] LogicGate class (__init__, feed_forward, loss_val)
class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4, 2) # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4, 1) # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        
        # 2층 hidden layer unit : 6개 가정. 가중치 W2. 바이어스 b2 초기화
        self.__W2 = np.random.rand(2, 6) # weight. 2 X 6 matrix
        self.__b2 = np.random.rand(6)
        
        # 3층 output layer unit : 1개. 가중치 W3. 바이어스 b3 초기화
        self.__W3 = np.random.rand(6, 1)
        self.__b3 = np.random.rand(1)
        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
        
        print(self.name + "object is created")
        
    def feed_forward(self): # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7 # log 무한대 발산 방지
        
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                 # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3          # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                            # 출력층의 출력
        
        # cross-entropy
        return -np.sum(self.__tdata*np.log(y + delta) + (1 - self.__tdata) * np.log(1 - y) + delta)
    
    def loss_val(self): # 외부 출력을 위한 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7 # log 무한대 발산 방지
        
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # 은익층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3           # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                             # 출력층의 출력
        
        # cross-entropy
        return -np.sum(self.__tdata*np.log(y + delta) + (1 - self.__tdata) * np.log((1 - y) + delta))
# [2] LogucGate class (train, predict)
    # 수치미분을 이용하여 손실함수가 최소가 될 때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in range(10001):
            
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
            
            if (step % 400 == 0):
                print("step = ", step, " , loss value = ", self.loss_val())
                
    # query. 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                          # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3    # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                      # 출력층의 출력
        
        if y > 0.5:
            result = 1 # True
        else:
            result = 0 # False
            
        return y, result

검증코드 - 딥러닝 버전

In [15]:
# [1] AND Gate 검증
# AND Gate 객체 생성 및 training

xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)

and_obj.train()

ANDobject is created
Initial loss value =  8.719647976602285
step =  0  , loss value =  8.408973368966324
step =  400  , loss value =  2.2504173993298977
step =  800  , loss value =  2.1271168709831407
step =  1200  , loss value =  1.9501162404936285
step =  1600  , loss value =  1.7046400172891165
step =  2000  , loss value =  1.409711567806226
step =  2400  , loss value =  1.1121650680364996
step =  2800  , loss value =  0.8546258410118889
step =  3200  , loss value =  0.6544255754108779
step =  3600  , loss value =  0.5075317249679467
step =  4000  , loss value =  0.4017682327327912
step =  4400  , loss value =  0.32524770314198637
step =  4800  , loss value =  0.268961264038719
step =  5200  , loss value =  0.22668402330468937
step =  5600  , loss value =  0.19423925007020149
step =  6000  , loss value =  0.16882739423356208
step =  6400  , loss value =  0.14854966097742978
step =  6800  , loss value =  0.13209571184803293
step =  7200  , loss value =  0.1185438107281859
step =  76

In [16]:
# AND Gate prediction

test_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

for data in test_data:
    print(and_obj.predict(data))

(array([0.00139813]), 0)
(array([0.01707189]), 0)
(array([0.01720771]), 0)
(array([0.97008859]), 1)


In [17]:
# [2] OR Gate 검증
# OR Gate 객체 생성 및 training

xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)

or_obj.train()

ORobject is created
Initial loss value =  2.941063118720334
step =  0  , loss value =  2.917729794460086
step =  400  , loss value =  1.9730170414729082
step =  800  , loss value =  1.7942871862882366
step =  1200  , loss value =  1.5324382556531815
step =  1600  , loss value =  1.2026750591505169
step =  2000  , loss value =  0.8791275869156017
step =  2400  , loss value =  0.6260360608614346
step =  2800  , loss value =  0.45170414856045904
step =  3200  , loss value =  0.3363060435821323
step =  3600  , loss value =  0.2592627612759465
step =  4000  , loss value =  0.20635416086645364
step =  4400  , loss value =  0.16879200943419456
step =  4800  , loss value =  0.14125429337345669
step =  5200  , loss value =  0.12047285648394707
step =  5600  , loss value =  0.10438728915927953
step =  6000  , loss value =  0.0916595377024241
step =  6400  , loss value =  0.08139489931196671
step =  6800  , loss value =  0.07297855582240498
step =  7200  , loss value =  0.06597734468150204
step =

In [18]:
# OR Gate prediction

test_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

for data in test_data:
    print(or_obj.predict(data))

(array([0.02257484]), 0)
(array([0.99243533]), 1)
(array([0.99250832]), 1)
(array([0.99970969]), 1)


In [19]:
# [3] NAND Gate 검증
# NAND Gate 객체 생성 및 training

xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

NANDobject is created
Initial loss value =  2.8320685058540107
step =  0  , loss value =  2.8131585309035074
step =  400  , loss value =  2.2718164342619165
step =  800  , loss value =  2.1561500196830687
step =  1200  , loss value =  1.9838172046904319
step =  1600  , loss value =  1.7028681281708131
step =  2000  , loss value =  1.3161180010649425
step =  2400  , loss value =  0.9684891193012679
step =  2800  , loss value =  0.7074423556954407
step =  3200  , loss value =  0.5218910443772673
step =  3600  , loss value =  0.39389345376916907
step =  4000  , loss value =  0.30569338591273487
step =  4400  , loss value =  0.24386015584296278
step =  4800  , loss value =  0.19941469532647937
step =  5200  , loss value =  0.1666061719292546
step =  5600  , loss value =  0.14176514705776827
step =  6000  , loss value =  0.122517199882763
step =  6400  , loss value =  0.10729284967078334
step =  6800  , loss value =  0.09503002097019553
step =  7200  , loss value =  0.08499302503193612
step

In [21]:
# NAND Gate prediction

test_data = np.array([[0, 0], [1, 0], [1, 0], [1, 1]])

for data in test_data:
    print(nand_obj.predict(data))

(array([0.99989062]), 1)
(array([0.9878995]), 1)
(array([0.9878995]), 1)
(array([0.02242803]), 0)


In [24]:
# [4] XOR Gate 검증
# XOR Gate 객체 생성 및 training

xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([0, 1, 1, 0])

xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XORobject is created
Initial loss value =  4.776081507459481
step =  0  , loss value =  4.675896219325046
step =  400  , loss value =  2.7605249598100543
step =  800  , loss value =  2.7523873509391317
step =  1200  , loss value =  2.7419722365639605
step =  1600  , loss value =  2.727838913843216
step =  2000  , loss value =  2.7079899368291946
step =  2400  , loss value =  2.679645672615663
step =  2800  , loss value =  2.639155843239216
step =  3200  , loss value =  2.582490705788363
step =  3600  , loss value =  2.5069161125790878
step =  4000  , loss value =  2.4133849204821223
step =  4400  , loss value =  2.307096468995857
step =  4800  , loss value =  2.1946462580148833
step =  5200  , loss value =  2.0802468963210865
step =  5600  , loss value =  1.964026744973213
step =  6000  , loss value =  1.8425707380345395
step =  6400  , loss value =  1.7107671528734794
step =  6800  , loss value =  1.5649157663053668
step =  7200  , loss value =  1.4065059520472736
step =  7600  , loss

In [25]:
test_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.10373105]), 0)
(array([0.88019993]), 1)
(array([0.87945738]), 1)
(array([0.15409941]), 0)


XOR Gate 검증 - 딥러닝 버전

- => 신경망 기반의 딥러닝을 구현하여 XOR 문제 해결 !!
- 1) NAND / OR / AND 조합을 이용하지 않고,
- 2) 입력층 / 은닉층 / 출력층으로 구성된 딥러닝 아키텍처 (Neural Network) 이용

- ==> 입력층, 1개 이상의 은닉층, 출력층을 가지는 딥러닝을 설계한다면, XOR 보다는 더 복잡한 필기체 손 글씨 인식, 이미지 인식 등의 문제도 해결 할 수 있다는 insight를 얻을 수 있음.